In [32]:
import json

In [33]:
import geopandas as gpd
import pandas as pd

In [34]:
import topojson as tp

## Get Census Data

In [35]:
census_df = pd.read_csv("data/slavery-map/census.csv")

## Open GIS Data

In [36]:
with open("data/slavery-map/us.json", "r") as f:
    us_geo_dict = json.load(f)

In [37]:
topo = tp.Topology(us_geo_dict, object_name="county_1790")
geojson = json.loads(topo.to_geojson())

In [39]:
for i, feature in enumerate(geojson["features"]):
    geojson["features"][i]["properties"]["id"] = feature["id"]

In [40]:
counties_gdf = gpd.GeoDataFrame.from_features(geojson)

## Join the Data

In [57]:
counties_gdf = counties_gdf.rename(columns={"id": "GISJOIN"})

In [58]:
counties_gdf

,geometry,c,s,GISJOIN
0,"MULTIPOLYGON (((700.031 240.983, 700.999 240.0...",Elizabeth City,Virginia,G5100550
1,"MULTIPOLYGON (((712.86 197.901, 711.076 199.59...",Cumberland,New Jersey,G3400110
2,"POLYGON ((590.082 241.371, 590.234 242.202, 59...",Fayette (old),Virginia,G5100617
3,"POLYGON ((633.929 192.159, 634.167 193.519, 63...",Ohio,Virginia,G5101355
4,"POLYGON ((714.854 160.784, 714.854 160.839, 71...",Orange,New York,G3600710
...,...,...,...,...
287,"POLYGON ((630.628 270.437, 631.263 271.783, 63...",Burke,North Carolina,G3700230
288,"POLYGON ((721.348 124.307, 720.972 124.275, 72...",Bennington,Vermont,G5000030
289,"MULTIPOLYGON (((702.306 191.05, 701.765 191.62...",New Castle,Delaware,G1000030
290,"POLYGON ((679.661 268.999, 679.957 269.168, 67...",Johnston,North Carolina,G3701010


In [65]:
census_df = census_df.iloc[:290]

In [66]:
counties_with_slavery_info = counties_gdf.merge(census_df, on="GISJOIN", how="right")

In [69]:
counties_with_slavery_info = counties_with_slavery_info.set_crs(5070)

In [70]:
counties_with_slavery_info.to_file("data/slavery.gpkg")